In [1]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()), 'research/qd/diffusion_models')
os.chdir(project_root)
%pwd # should be PPGA root dir

'/home/shashank/research/qd/diffusion_models'

In [2]:
from autoencoders.policy_hyperautoencoder import HyperAutoEncoder
from envs.brax_custom.brax_env import make_vec_env_brax

from dataset.policy_dataset import e_data_loader_test, e_data_loader_train, env_cfg, actor_cfg

/home/shashank/miniconda3/envs/qd/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 791/791 [00:02<00:00, 369.68it/s]


In [3]:
from dataset.policy_dataset import actor_cfg, Actor, test_elites, scheduler
import torch
from IPython.display import HTML, Image
from brax.io import html, image
import torch.nn.functional as F


In [4]:
env = make_vec_env_brax(env_cfg)

In [5]:
model = HyperAutoEncoder(actor_cfg, emb_channels=8, z_channels=4)
model_cp = './checkpoints/autoencoder.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.load_state_dict(torch.load(model_cp))
model.to(device)
# model.eval()

In [7]:
state_dict = torch.load(model_cp)
ghn_dict = {}
for k, v in state_dict.items():
    if 'decoder' in k:
        # print(".".join(k.split('.')[1:]))
        # print(k, v.shape)
        ghn_dict[".".join(k.split('.')[1:])] = v

model.load_state_dict(state_dict)
model.decoder.load_state_dict(ghn_dict)
model.to(device)
model = model.eval()


In [6]:
def enjoy_brax(agent, render=True, deterministic=True):
    if actor_cfg.normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(env_cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if actor_cfg.normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)

            if deterministic:
                act = agent.actor_mean(obs)
            else:
                act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew
    if render:
        i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
        display(i)
    print(f'{total_reward=}')
    # print(f' Rollout length: {len(rollout)}')
    measures /= len(rollout)
    print(f'Recorded Measures: {measures.cpu().numpy()}')
    return total_reward.detach().cpu().numpy()

In [7]:
# agent = Actor(actor_cfg, obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape, deterministic=True).to(device)
actor_cfg.normalize_obs

True

In [8]:
def get_random_elite():
    elite = scheduler.archive.sample_elites(1)
    agent = Actor(actor_cfg, obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape).deserialize(elite.solution_batch.flatten()).to(device)
    if actor_cfg.normalize_obs:
        agent.obs_normalizer = elite.metadata_batch[0]['obs_normalizer']
    return agent, elite.measures_batch[0], elite

In [6]:
agent, measure, elite = get_random_elite()
print(f'Sampled measure from elite: {measure}')


NameError: name 'get_random_elite' is not defined

In [46]:
enjoy_brax(agent, render=False)

total_reward=tensor(3062.0259, device='cuda:0')
Recorded Measures: [0.7422577  0.84415585]


array(3062.026, dtype=float32)

In [28]:
# weights_dict = agent.get_deserialized_weights(elite.solution_batch.flatten())
# sampled_x,y = e_data_loader_test.__iter__().__next__()
# x.keys()

In [47]:
# x_ = agent.state_dict()
x = {}
for key, param in agent.named_parameters():
    if 'weight' in key or 'bias' in key or 'logstd' in key:
        x[key] = param.unsqueeze(0)
# x

In [48]:
reconstructed_models = model(x)

In [49]:
pred_weights_dict = {}
for out_ in reconstructed_models:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), x[key])

mse_loss

tensor(0.1269, device='cuda:0', grad_fn=<AddBackward0>)

In [21]:
# for i, recon_model in enumerate(reconstructed_models[0]):
    # desired_measure = y[i]
    # desired_model_params = {key:x[key][i] for key in x.keys() }
    # print(f'Desired measure: {desired_measure}')
    # agent.actor_mean = desired_model
    # agent.actor_mean.load_state_dict(desired_model_params)
agent.actor_mean = reconstructed_models[0].actor_mean
# agent.actor_mean = reconstructed_models[0][0].actor_mean

enjoy_brax(agent, render=False)
print('--------------------------------------------------------------------')

total_reward=tensor(-8.8833, device='cuda:0')
Recorded Measures: [0.996004   0.99900097]
--------------------------------------------------------------------


In [8]:
sampled_x,y = e_data_loader_train.__iter__().__next__()
img_out = model(sampled_x)
# loss = loss_func(batch, img_out, posterior, global_step, 0)
# loss += loss_func(batch, img_out, posterior, global_step, 1)
pred_weights_dict = {}
for out_ in img_out:
    for name, param in out_.named_parameters():
        if name not in pred_weights_dict:
            pred_weights_dict[name] = []
        pred_weights_dict[name].append(param)
        

# loss is MSE loss
mse_loss = 0
for key in pred_weights_dict:
    mse_loss += F.mse_loss(torch.stack(pred_weights_dict[key]), sampled_x[key])

mse_loss

tensor(0.1047, device='cuda:0', grad_fn=<AddBackward0>)

In [11]:
new_state_dict = model.state_dict()
for key in new_state_dict:
    new_state_dict[key] == state_dict[key]